In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import os
google_colab = input('Running on Colab?\n')
if google_colab=='yes' : 
    BASE_DIR = 'University/UC3M/TFM/cnp_repo/neural-processes'

    # Load the Drive helper and mount
    from google.colab import drive

    # This will prompt for authorization.
    drive.mount('/content/drive')

      # Change to assignment directory ('Lab_Exercises_DASS/Lab_Exercise4' by default)
    import os
    os.chdir(f'/content/drive/My Drive/{BASE_DIR}')
    
else:
    import sys, os 
    BASE_DIR = '/'.join(os.getcwd().split('/')[:-1])
    sys.path.append(BASE_DIR)
    print(f'Top level directory is set to {BASE_DIR}')
    os.chdir(BASE_DIR)

    
# custom imports
from cnp.trainer import RegressionTrainer
from cnp.datageneration import DataGenerator
from cnp.helpers import Helper

import torch
import numpy as np
from torch import nn


# import utils
import json
from datetime import date

Running on Colab?
o
Top level directory is set to /Users/frederik/Google Drive/University/UC3M/TFM/cnp_repo/neural-processes


In [3]:

# we first get the variables for the configuration file
os.environ["CHECKPOINT_DIR"] = "checkpoints"
os.environ["MODEL_PARAMS"] = "default"

CHECKPOINT_DIR = os.environ['CHECKPOINT_DIR']
MODEL_PARAMS = os.environ['MODEL_PARAMS']

if google_colab=='yes':
    config_dir = ''
else:
    config_dir = os.path.join(BASE_DIR,'configs')

with open(os.path.join(config_dir,'1dpolynomial_fixedcontxt_mlpaggr.json')) as f:
    file = f.read()
    default_conf = json.loads(file)



In [4]:

# checking whether CUDA is available 
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
    default_conf['experiment_params']['train_on_gpu'] = True
else: 
    print('No GPU available, training on CPU')
    default_conf['experiment_params']['train_on_gpu'] = False

No GPU available, training on CPU


## We will first try to replicate what we have after one encoding step

In [51]:
import pandas as pd 
import numpy as np
import sys
import matplotlib.pyplot as plt
from cnp.networks import Encoder, create_linear_layer
from cnp.cnp import get_sample_indexes, select_data
from sklearn.linear_model import LinearRegression
from cnp.datageneration import PolynomialRegression, DataGenerator

In [50]:
default_conf

{'cnp_params': {'dimout': 2,
  'dimr': 128,
  'dimx': 1,
  'dimy': 1,
  'dropout': 0,
  'max_contx': 10,
  'max_funcs': 10,
  'min_contx': 3,
  'min_funcs': 2,
  'fix_num_contxt': True,
  'num_layers_decoder': 3,
  'num_layers_encoder': 4,
  'num_neurons_decoder': 128,
  'num_neurons_encoder': 128,
  'aggregation_kwargs': {'aggregator': 'MLPAggregator',
   'insize': 5,
   'dimout': 1,
   'num_layers': 2,
   'num_neurons': 20,
   'padding': False}},
 'data_kwargs': {'datagenerator': 'cnp.datageneration.PolynomialRegression',
  'mu': 0,
  'sigma': 2,
  'num_instances_train': 64,
  'num_instances_vali': 1,
  'num_instances_test': 1},
 'experiment_params': {'dim_observation': 50,
  'lr': 0.001,
  'n_epochs': 10000,
  'range_x': [-2, 2],
  'seed': None,
  'dimx': 1,
  'train_on_gpu': False},
 'train_kwargs': {'batch_size_train': 64,
  'batch_size_vali': 1,
  'batch_size_test': 1,
  'plotting': True,
  'print_after': 100}}

In [55]:
datagen = DataGenerator(xdim=5)

In [53]:
dir(datagen)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_create_x',
 '_steps',
 '_xdim',
 '_xmax',
 '_xmin',
 'generate_curves',
 'generate_from_single_function',
 'generate_loader_on_fly']

In [61]:
datagen._create_x()[1]

tensor([-1.9900, -1.9900, -1.9900, -1.9900, -1.9900])

In [ ]:
def create_function(num_polynomials=2,num_dims=2):
    x_data = torch.linspace(-2, 2,400)
    x_data = x_data.unsqueeze(-1)
    x_data = x_data.repeat(1, num_dims)
    
    